In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6008...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': 8
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


0
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553085918943
snopt_example=[ 0.28673366 -0.30128467  0.33402563], growth = 0.09485392272232604
snopt_example=[ 0.28776043 -0.30045108  0.32725976], growth = 0.09339195802962709
terminating because a required containment point would have not been contained
Time:   3.00 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
0
snopt_example=[ 0.35529266 -0.22984424  0.02559962], growth = 0.059905530775535676
snopt_example=[ 0.30804713 -0.27858144  0.24207296], growth = 0.07703332293708191
terminating because a required containment point would have not been contained
Time:   0.57 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
snopt_example=[ 0.30547805 -0.5206824   0.30020754], growth = 354169277.0183215
snopt_example=[ 0.19754042 -0.45436384  0.26846798], growth = 33224401.492606163
snopt_example=[ 0.19598982 -0.45413508  0.27106353], growth = 32414169.416864872
0
snopt_example

In [9]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [10]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 70)
    

## Certify the regions

In [18]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

Time to initialize region program: 0.7936
time to create Region Certification Problem: : 0.4034
Time to initialize region program: 0.7565
time to create Region Certification Problem: : 0.4008
Time to initialize region program: 0.8389
time to create Region Certification Problem: : 0.7546
Time to initialize region program: 0.7483
time to create Region Certification Problem: : 0.3982
Time to initialize region program: 0.8364
time to create Region Certification Problem: : 0.7555
Time to initialize region program: 0.8068
time to create Region Certification Problem: : 0.6259


{<pydrake.geometry.optimization.HPolyhedron at 0x7ff281e03f70>: <certified_iris_generator.RegionCertificationProblem at 0x7ff2982934f0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7ff2a07102f0>: <certified_iris_generator.RegionCertificationProblem at 0x7ff233ea2f10>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7ff281e069b0>: <certified_iris_generator.RegionCertificationProblem at 0x7ff2a0704dc0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7ff281e10d30>: <certified_iris_generator.RegionCertificationProblem at 0x7ff298293490>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7ff2a06da070>: <certified_iris_generator.RegionCertificationProblem at 0x7ff233ea2340>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7ff281e10b70>: <certified_iris_generator.RegionCertificationProblem at 0x7ff233ea2040>}

In [19]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/6
min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 3.317209025457233
Smallest difference: 0.14689258966773977
Fixed eps is not success

min_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
cur_eps = [1.29903811 2.48790677 0.16248491 1.29903811 0.11016944 2.4355913 ]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 1.6586045127286164
Smallest difference: 0.07344629483386989
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.62540447 1.24395338 0.08124245 0.46851097 0.05508472 1.21779565]
max_eps = [1.25080894 2.48790677 0.1624849  0.93702194 0.11016943 2.4355913 ]
Biggest difference: 2.4879067681090223
Smallest difference: 0.11016943481040004
Fixed eps is not success

min_eps = [0.62540447 1.24395338 0.081

Fixed eps is not success

min_eps = [0.98321771 2.4347021  0.15901007 0.2628375  0.1078134  2.38350542]
cur_eps = [0.99102102 2.45402514 0.16027206 0.26492352 0.10866906 2.40242213]
max_eps = [0.99882434 2.47334817 0.16153404 0.26700953 0.10952472 2.42133884]
Biggest difference: 0.03864606514971003
Smallest difference: 0.0017113238063718311
Fixed eps is not success

min_eps = [0.99102102 2.45402514 0.16027206 0.26492352 0.10866906 2.40242213]
cur_eps = [0.99492268 2.46368665 0.16090305 0.26596652 0.10909689 2.41188048]
max_eps = [0.99882434 2.47334817 0.16153404 0.26700953 0.10952472 2.42133884]
Biggest difference: 0.019323032574855237
Smallest difference: 0.0008556619031859086
Fixed eps is not success

min_eps = [0.99492268 2.46368665 0.16090305 0.26596652 0.10909689 2.41188048]
cur_eps = [0.99687351 2.46851741 0.16121854 0.26648802 0.10931081 2.41660966]
max_eps = [0.99882434 2.47334817 0.16153404 0.26700953 0.10952472 2.42133884]
Biggest difference: 0.00966151628742784
Smallest diff

Fixed eps is not success

min_eps = [1.74436424 2.46786399 1.13060524 0.7353731  0.13371111 1.35626964]
cur_eps = [1.86896169 2.64413998 1.21136276 0.78789975 0.1432619  1.45314604]
max_eps = [1.99355913 2.82041598 1.29212027 0.8404264  0.1528127  1.55002244]
Biggest difference: 0.35255199788927927
Smallest difference: 0.019101587238329443
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.93448084 1.28693303 0.60568138 0.31665309 0.07163095 0.59781055]
max_eps = [1.86896169 2.57386607 1.21136275 0.63330618 0.1432619  1.19562109]
Biggest difference: 2.573866065355878
Smallest difference: 0.14326190458656418
Fixed eps is not success

min_eps = [0.93448084 1.28693303 0.60568138 0.31665309 0.07163095 0.59781055]
cur_eps = [1.40172126 1.93039955 0.90852206 0.47497964 0.10744643 0.89671582]
max_eps = [1.86896169 2.57386607 1.21136275 0.63330618 0.1432619  1.19562109]
Biggest difference: 1.286933032677939
Smallest difference: 0.07163095229328209
Fixed eps

Fixed eps is not success

min_eps = [1.74071235 2.21115752 0.75156099 0.58984808 0.13343109 0.59982464]
cur_eps = [1.75452752 2.22870639 0.75752576 0.59452942 0.13449007 0.60458515]
max_eps = [1.7683427  2.24625526 0.76349053 0.59921075 0.13554904 0.60934566]
Biggest difference: 0.0350977384166522
Smallest difference: 0.002117953784214033
Fixed eps is not success

min_eps = [1.75452752 2.22870639 0.75752576 0.59452942 0.13449007 0.60458515]
cur_eps = [1.76143511 2.23748082 0.76050815 0.59687008 0.13501955 0.60696541]
max_eps = [1.7683427  2.24625526 0.76349053 0.59921075 0.13554904 0.60934566]
Biggest difference: 0.017548869208326323
Smallest difference: 0.0010589768921070164
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.88071754 1.11485872 0.28595771 0.29843502 0.06750973 0.19949114]
max_eps = [1.76143509 2.22971743 0.57191542 0.59687004 0.13501946 0.39898227]
Biggest difference: 2.2297174344177426
Smallest difference: 0.13501945608011898
Fixe

Fixed eps is not success

min_eps = [1.31914176 1.62019573 0.15225399 0.44419681 0.10111582 0.08731612]
cur_eps = [1.53899872 1.89022835 0.17762966 0.51822962 0.11796846 0.1018688 ]
max_eps = [1.75885568 2.16026097 0.20300532 0.59226242 0.13482109 0.11642149]
Biggest difference: 0.5400652434629034
Smallest difference: 0.029105372513108907
Fixed eps is not success

min_eps = [1.53899872 1.89022835 0.17762966 0.51822962 0.11796846 0.1018688 ]
cur_eps = [1.6489272  2.02524466 0.19031749 0.55524602 0.12639477 0.10914515]
max_eps = [1.75885568 2.16026097 0.20300532 0.59226242 0.13482109 0.11642149]
Biggest difference: 0.2700326217314517
Smallest difference: 0.014552686256554453
Fixed eps is not success

min_eps = [1.6489272  2.02524466 0.19031749 0.55524602 0.12639477 0.10914515]
cur_eps = [1.70389144 2.09275282 0.1966614  0.57375422 0.13060793 0.11278332]
max_eps = [1.75885568 2.16026097 0.20300532 0.59226242 0.13482109 0.11642149]
Biggest difference: 0.13501631086572585
Smallest differenc

Fixed eps is not success

min_eps = [0.15874691 0.22095344 0.14980417 0.17975223 0.11610076 0.20378539
 0.0002807  0.00591042 0.0058379 ]
cur_eps = [0.16403847 0.22831856 0.15479764 0.18574397 0.11997079 0.21057824
 0.00029005 0.00610743 0.00603249]
max_eps = [0.16933004 0.23568367 0.15979111 0.19173571 0.12384081 0.21737108
 0.00029941 0.00630445 0.00622709]
Biggest difference: 0.01473022958148168
Smallest difference: 1.871308850437864e-05
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.08201923 0.11415899 0.07739868 0.08982271 0.04680633 0.10528912
 0.00014307 0.00305372 0.00301617]
max_eps = [0.16403845 0.22831798 0.15479737 0.17964543 0.09361266 0.21057824
 0.00028613 0.00610744 0.00603234]
Biggest difference: 0.22831797607219173
Smallest difference: 0.00028613316390573755
Fixed eps is not success

min_eps = [0.08201923 0.11415899 0.07739868 0.08982271 0.04680633 0.10528912
 0.00014307 0.00305372 0.00301617]
cur_eps = [0.12302884 0.1

Fixed eps is not success

min_eps = [1.56816975 2.65102978 1.81349982 1.81349982 0.68569759 1.56816975]
cur_eps = [1.56855581 2.65168242 1.81394628 1.81394628 0.6858664  1.56855581]
max_eps = [1.56894187 2.65233506 1.81439273 1.81439273 0.68603521 1.56894187]
Biggest difference: 0.0013052830019728567
Smallest difference: 0.0003376157515225797
Fixed eps is success

min_eps = [1.56816975 2.65102978 1.81349982 1.81349982 0.68569759 1.56816975]
cur_eps = [1.56836278 2.6513561  1.81372305 1.81372305 0.685782   1.56836278]
max_eps = [1.56855581 2.65168242 1.81394628 1.81394628 0.6858664  1.56855581]
Biggest difference: 0.0006526415009866504
Smallest difference: 0.00016880787576123435
Fixed eps is not success

min_eps = [1.56836278 2.6513561  1.81372305 1.81372305 0.685782   1.56836278]
cur_eps = [1.56845929 2.65151926 1.81383466 1.81383466 0.6858242  1.56845929]
max_eps = [1.56855581 2.65168242 1.81394628 1.81394628 0.6858664  1.56855581]
Biggest difference: 0.00032632075049310316
Smallest d

Fixed eps is not success

min_eps = [0.65274595 1.0953723  0.82389388 0.73587896 0.07640635 0.56842353
 0.02624334 0.09619902]
cur_eps = [0.69937066 1.17361318 0.88274345 0.78844174 0.08186395 0.60902521
 0.02811787 0.10307037]
max_eps = [0.74599537 1.25185406 0.94159301 0.84100452 0.08732154 0.64962689
 0.02999239 0.10994173]
Biggest difference: 0.15648175770699524
Smallest difference: 0.003749049131502525
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.27266641 0.31633091 0.44137156 0.38194953 0.04093197 0.23215534
 0.01405893 0.05153519]
max_eps = [0.54533281 0.63266182 0.88274313 0.76389907 0.08186395 0.46431068
 0.02811787 0.10307037]
Biggest difference: 0.8827431267278688
Smallest difference: 0.028117868971843155
Fixed eps is not success

min_eps = [0.27266641 0.31633091 0.44137156 0.38194953 0.04093197 0.23215534
 0.01405893 0.05153519]
cur_eps = [0.40899961 0.47449637 0.66205735 0.5729243  0.06139796 0.34823301
 0.0210884  0.0773027

Fixed eps is not success

min_eps = [0.01579077 0.01539911 0.80988232 0.68266254 0.07791407 0.01750757
 0.02676121 0.09809731]
cur_eps = [0.01580622 0.01541418 0.81067476 0.68333051 0.07799031 0.0175247
 0.02678739 0.09819329]
max_eps = [0.01582167 0.01542925 0.81146721 0.68399848 0.07806655 0.01754183
 0.02681358 0.09828928]
Biggest difference: 0.0015848968991971635
Smallest difference: 3.0135246166037785e-05
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.00657624 0.00645095 0.39660165 0.32432292 0.03899515 0.0071939
 0.0133937  0.04909665]
max_eps = [0.01315248 0.01290189 0.7932033  0.64864583 0.07799031 0.0143878
 0.02678739 0.09819329]
Biggest difference: 0.7932033012129371
Smallest difference: 0.012901894942746406
Fixed eps is not success

min_eps = [0.00657624 0.00645095 0.39660165 0.32432292 0.03899515 0.0071939
 0.0133937  0.04909665]
cur_eps = [0.00986436 0.00967642 0.59490248 0.48648438 0.05849273 0.01079085
 0.02009054 0.0736449

In [20]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [14]:
do_alt_cert = False
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 1000, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

In [21]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [22]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(trajectory_start.squeeze(), trajectory_end.squeeze(), 
                                                iris_generator.linesearch_regions)
spp.set_max_velocity(0.3*np.ones(plant.num_positions()))
spp.set_extra_control_points_per_region(3)
traj_iris = spp.SolveVerbose()
visualizer.draw_traj_tspace(traj_iris, 100, 'iris')

In [30]:
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
# draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 10
visualizer.animate_t(traj_iris, substeps, its*substeps)

173.6671793460846
0.0
4.501785276458669
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PR

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

...

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 

THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A BIG PROBLEM
THERE IS A 